<a href="https://colab.research.google.com/github/murthylab/sleap-notebooks/blob/master/Training_and_inference_on_an_example_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we'll show you how to install SLEAP on Colab, download a dataset from the repository of sample datasets, run training and inference on that dataset using the SLEAP command-line interface, and then download the predictions.

# Installation

Before we install SLEAP we need to set Colab to use TensorFlow 2.

In [0]:
%tensorflow_version 2.x

Let's confirm that we have a GPU available. This next line should return something like "/device:GPU:0". If you instead see an empty string as the result, go to "Notebook settings" in the "Edit" menu and select "GPU" as the hardware accelerator.

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

''

Since Colab already has TensorFlow and GPU drivers installed we can install SLEAP using `pip`.

Note: This installation method should also work on other Linux machines, such as an HPC cluster, or on any system where you aren't planning to use a GPU. To use a GPU on a Windows machine you'll need to install using `conda`.

In [0]:
!pip install sleap==1.0.10

## Getting your training data into Colab

You'll need to get your training data into Colab. The easiest way to do this is to export a self-contained **training package** from SLEAP and copy that onto [Google Drive](https://www.google.com/drive).

For this demo we'll instead download a sample dataset from the SLEAP [sample datasets repository](https://github.com/murthylab/sleap-datasets).

In [0]:
!apt-get install tree
!wget -O dataset.zip https://github.com/murthylab/sleap-datasets/releases/download/dm-courtship-v1/drosophila-melanogaster-courtship.zip
!mkdir dataset
!unzip dataset.zip -d dataset
!rm dataset.zip
!tree dataset

# Training

Now you're ready to train a model! We'll use the command-line interface for training, and train a model for confidence maps using the default **training profile**. The training profile determines the model architecture, the learning rate for training, and other training parameters.

When you start running this cell, you'll see the training parameters listed and then you'll see the training and validation loss for each training epoch.

If you're happy with the validation loss you see for an epoch during training, you're welcome to stop training by clicking the stop button next to the notebook cell running training. The version of the model with the lowest validation loss is saved during training, and that's what will be used for inference. If you don't stop training, it will run for 200 epochs or until validation loss fails to improve for some number of epochs (controlled by the `early_stopping` fields in the training profile).

In [0]:
!sleap-train baseline.centroid.json "dataset/drosophila-melanogaster-courtship/courtship_labels.slp" --run_name "courtship.centroid" --video-paths "dataset/drosophila-melanogaster-courtship/20190128_113421.mp4"

Let's also train a centered instance model since we'll need both a centroid model and a top-down confidence maps model to run inference with the top-down pipeline.

In [0]:
!sleap-train baseline_medium_rf.topdown.json "dataset/drosophila-melanogaster-courtship/courtship_labels.slp" --run_name "courtship.topdown_confmaps" --video-paths "dataset/drosophila-melanogaster-courtship/20190128_113421.mp4"

The models (along with the profiles and ground truth data used to train and validate the model) are saved in the `models/` directory:

In [0]:
!tree models/

models/
├── courtship.centroid
│   ├── best_model.h5
│   ├── initial_config.json
│   ├── labels_gt.train.slp
│   ├── labels_gt.val.slp
│   ├── training_config.json
│   └── training_log.csv
└── courtship.topdown_confmaps
    ├── best_model.h5
    ├── initial_config.json
    ├── labels_gt.train.slp
    ├── labels_gt.val.slp
    ├── training_config.json
    └── training_log.csv

2 directories, 12 files


# Inference

At this point you should have SLEAP installed, the sample dataset downloaded, and trained models for centroids and topdown confidence maps.

In [0]:
!sleap-track "dataset/drosophila-melanogaster-courtship/20190128_113421.mp4" --frames 0-100 -m "models/courtship.centroid" -m "models/courtship.topdown_confmaps"

When inference is finished, it will save the predictions in a file. Since we didn't specify a path, it will be saved as `<video filename>.predictions.slp` in the same directory as the video:

In [0]:
!tree dataset/drosophila-melanogaster-courtship

dataset/drosophila-melanogaster-courtship
├── 20190128_113421.mp4
├── 20190128_113421.mp4.predictions.slp
├── courtship_labels.slp
└── example.jpg

0 directories, 4 files


You can inspect your predictions file using `sleap-inspect`:

In [0]:
!sleap-inspect dataset/drosophila-melanogaster-courtship/20190128_113421.mp4.predictions.slp

If you're using Chrome you can download your trained models like so:

In [0]:
# Zip up the models directory
!zip -r trained_models.zip models/

# Download.
from google.colab import files
files.download("/content/trained_models.zip")

And you can likewise download your predictions:

In [0]:
from google.colab import files
files.download('dataset/drosophila-melanogaster-courtship/20190128_113421.mp4.predictions.slp')

In some other browsers (Safari) you might get an error and you can instead download using the "Files" tab in the side panel (it has a folder icon). Select "Show table of contents" in the "View" menu if you don't see the side panel.